## v_student_info

In [0]:
CREATE OR REPLACE VIEW v_student_info AS
SELECT
  si.id_student,
  si.gender,
  si.age_band,
  si.highest_education,
  si.region,
  si.final_result,
  sr.code_module,
  sr.code_presentation
FROM
  student_info si
JOIN 
  student_registration_filtered sr
ON 
  si.id_student = sr.id_student;

In [0]:
SELECT *
FROM
  v_student_info
LIMIT 10;

id_student,gender,age_band,highest_education,region,final_result,code_module,code_presentation
11391,M,55<=,HE Qualification,East Anglian Region,Pass,AAA,2013J
28400,F,35-55,HE Qualification,Scotland,Pass,AAA,2013J
30268,F,35-55,A Level or Equivalent,North Western Region,Withdrawn,AAA,2013J
31604,F,35-55,A Level or Equivalent,South East Region,Pass,AAA,2013J
32885,F,0-35,Lower Than A Level,West Midlands Region,Pass,AAA,2013J
38053,M,35-55,A Level or Equivalent,Wales,Pass,AAA,2013J
45462,M,0-35,HE Qualification,Scotland,Pass,AAA,2013J
45642,F,0-35,A Level or Equivalent,North Western Region,Pass,AAA,2013J
52130,F,0-35,A Level or Equivalent,East Anglian Region,Pass,AAA,2013J
53025,M,55<=,Post Graduate Qualification,North Region,Pass,AAA,2013J


## v_vle_summary

In [0]:
CREATE OR REPLACE VIEW v_vle_summary AS
SELECT
  id_student,
  code_module,
  code_presentation,
  SUM(sum_click) AS total_clicks,
  COUNT(DISTINCT date) AS days_active
FROM
  student_vle
GROUP BY
  id_student,
  code_module,
  code_presentation;



SELECT * 
FROM
  v_vle_summary
LIMIT 10;

id_student,code_module,code_presentation,total_clicks,days_active
505131,FFF,2014B,943,54
585572,FFF,2014J,2791,81
594229,FFF,2014J,3707,82
643690,FFF,2014J,395,18
2337889,FFF,2013J,4175,94
576518,FFF,2013J,3360,84
585449,FFF,2013J,2638,102
625828,FFF,2014B,610,32
1615392,FFF,2014B,4692,152
630177,FFF,2014B,3825,174


## v_assessment_score

In [0]:
CREATE OR REPLACE VIEW v_assessment_score AS
SELECT
  sa.id_student,
  sr.code_module,
  sr.code_presentation,
  COUNT(*) AS num_submissions,
  ROUND(AVG(sa.score),2) AS avg_score
FROM
  student_assessment_filtered sa
JOIN
  student_registration_filtered sr
ON 
  sa.id_student = sr.id_student
GROUP BY
  sa.id_student,
  sr.code_module,
  sr.code_presentation;


SELECT *
FROM
  v_assessment_score
LIMIT 10;

id_student,code_module,code_presentation,num_submissions,avg_score
396655,AAA,2014J,5,40.2
528464,BBB,2013J,11,65.82
553078,BBB,2013J,10,81.5
556427,BBB,2013J,6,95.17
626126,BBB,2014B,10,85.3
626138,BBB,2014B,11,90.45
648310,BBB,2014J,5,65.4
84299,FFF,2014B,17,75.47
597403,DDD,2013J,14,70.14
512475,CCC,2014B,5,93.4


## v_student_timeline

In [0]:
CREATE OR REPLACE VIEW v_student_timeline AS
SELECT
  id_student,
  code_module,
  code_presentation,
  real_date_registration,
  real_date_unregistration,
  CASE
    WHEN real_date_unregistration IS NOT NULL THEN DATEDIFF(real_date_unregistration, real_date_registration)
    ELSE
     250
  END AS learning_days
FROM student_registration_filtered;

SELECT *
FROM
  v_student_timeline
LIMIT 10;

id_student,code_module,code_presentation,real_date_registration,real_date_unregistration,learning_days
11391,AAA,2013J,2012-07-26,null,250
28400,AAA,2013J,2012-11-09,null,250
30268,AAA,2013J,2012-10-01,2013-01-13,104
31604,AAA,2013J,2012-11-10,null,250
32885,AAA,2013J,2012-07-09,null,250
38053,AAA,2013J,2012-09-13,null,250
45462,AAA,2013J,2012-10-26,null,250
45642,AAA,2013J,2012-12-03,null,250
52130,AAA,2013J,2012-11-29,null,250
53025,AAA,2013J,2012-07-06,null,250


## v_student_summary

In [0]:
CREATE OR REPLACE VIEW v_student_summary AS
SELECT
  info.id_student,
  info.code_module,
  info.code_presentation,
  info.gender,
  info.age_band, 
  info.highest_education,
  info.final_result,
  vs.total_clicks,
  vs.days_active,
  ass.num_submissions,
  ass.avg_score,
  tl.real_date_registration,
  tl.real_date_unregistration,
  tl.learning_days
FROM v_student_info info
LEFT JOIN v_vle_summary vs
  ON info.id_student = vs.id_student
  AND info.code_module = vs.code_module
  AND info.code_presentation = vs.code_presentation
LEFT JOIN v_assessment_score ass
  ON info.id_student = ass.id_student
  AND info.code_module = ass.code_module
  AND info.code_presentation = ass.code_presentation
LEFT JOIN v_student_timeline tl
  ON info.id_student = tl.id_student
  AND info.code_module = tl.code_module
  AND info.code_presentation = tl.code_presentation;


SELECT *
FROM
  v_student_summary
LIMIT 10;


id_student,code_module,code_presentation,gender,age_band,highest_education,final_result,total_clicks,days_active,num_submissions,avg_score,real_date_registration,real_date_unregistration,learning_days
2228010,BBB,2014J,F,35-55,Lower Than A Level,Pass,2052,73,5,63.0,2013-11-26,null,250
133816,AAA,2014J,F,35-55,A Level or Equivalent,Pass,4000,128,5,65.4,2013-12-17,null,250
540513,BBB,2013B,F,0-35,A Level or Equivalent,Pass,462,27,11,79.64,2013-03-18,null,250
573893,BBB,2013J,F,35-55,HE Qualification,Fail,662,45,11,76.27,2012-10-26,null,250
588321,BBB,2013J,F,35-55,Lower Than A Level,Withdrawn,238,27,7,66.86,2012-09-07,2013-06-10,276
555720,BBB,2014B,F,0-35,Lower Than A Level,Fail,8,5,3,44.0,2013-12-28,null,250
381909,BBB,2014B,F,35-55,A Level or Equivalent,Distinction,1756,78,11,89.64,2014-04-07,null,250
1840034,BBB,2014B,F,35-55,Lower Than A Level,Pass,376,50,10,81.9,2014-04-07,null,250
185057,EEE,2014B,M,35-55,HE Qualification,Withdrawn,3542,155,5,66.4,2013-12-29,null,250
617804,BBB,2014J,M,0-35,A Level or Equivalent,Withdrawn,null,null,null,null,2013-10-16,2013-11-16,31
